# Исследование надёжности заёмщиков банка
**Проектная работа №1 Яндекс.Практикум - Data Science**

### Описание проекта

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок.

**Исходные данные:**

Статистика о платёжеспособности клиентов и данные о клиентах.

**Цели проекта:**

Подготовить результаты исследования для построения модели кредитного скоринга.

**Условия проекта:**

Необходимо изучить и обработать полученные входные данные, и ответить на вопросы:
* Есть ли зависимость между наличием детей и возвратом кредита в срок?
* Есть ли зависимость между семейным положением и возвратом кредита в срок?
* Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
* Как разные цели кредита влияют на его возврат в срок?

### Структура проекта
* 1. [Загрузка и изучение общей информации датасета](#start)
* 2. [Предобработка данных](#preprocessing)
    * 2.1 [Обработка пропущенных значений](#handling_missing_values)
    * 2.2 [Замена вещественного типа данных на целочисленный](#float_to_int)
    * 2.3 [Обработка дубликатов](#duplicates)
    * 2.4 [Выделение леммы в значениях столбца с целями получения кредита](#lemma)
    * 2.5 [Категоризация данных](#categories)
* 3. [Ответы на вопросы исследования](#answers)
    * 3.1 [Есть ли зависимость между наличием детей и возвратом кредита в срок?](#answer_1)
    * 3.2 [Есть ли зависимость между семейным положением и возвратом кредита в срок?](#answer_2)
    * 3.3 [Есть ли зависимость между уровнем дохода и возвратом кредита в срок?](#answer_3)
    * 3.4 [Как разные цели кредита влияют на его возврат в срок?](#answer_4)
* 4. [Общий вывод](#conclusion)

<a id="start"></a>
## 1. Загрузка и изучение общей информации датасета

#### Импортируем необходимые библиотеки

In [124]:
import pandas as pd
import sys
from pymystem3 import Mystem
from nltk.stem import SnowballStemmer
from collections import Counter

#### Загрузим данные

In [125]:
dataset = 'data.csv'

try:
    df = pd.read_csv(f'./datasets/{dataset}', sep=',')
    print(f'Прочитан файл с данными: "./datasets/{dataset}"')
except:
    try:
        df = pd.read_csv(f'/datasets/{dataset}', sep=',') # yandex.praktikum
        print(f'Прочитан файл с данными: "/datasets/{dataset}"')
    except Exception as err:
        print(repr(err))

Прочитан файл с данными: "./datasets/data.csv"


#### Узнаём типы данных, количество непустых значений, размерность и общий размер датасета

In [126]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB
None


#### Напечатаем первые 5 строк DataFrame

In [127]:
df.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


##### Вывод:
- датасет на 21525 строк по 12 значений
- размер датасета 2.0+ MB
- типы данных в датасете верные
- в столбцах 'days_employed' и 'total_income' присутствуют пропущенные значения

<a id="preprocessing"></a>
## 2. Предобработка данных

<a id="handling_missing_values"></a>
### 2.1 Обработка пропущенных значений

#### Узнаем кол-во пропущенных значений методом isna()

In [128]:
print('Количество пропущенных значений в столбке \'days_employed\':', sum(df['days_employed'].isna()))
print('Количество пропущенных значений в столбке \'total_income\':', sum(df['total_income'].isna()))

Количество пропущенных значений в столбке 'days_employed': 2174
Количество пропущенных значений в столбке 'total_income': 2174


#### Проверим зависимость пропущенных значений в 'days_employed' и в 'total_income' друг от друга

In [129]:
df[(df['days_employed'].isna()) | (df['total_income'].isna())].head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу


In [130]:
df[(df['days_employed'].isna()) | (df['total_income'].isna())].tail(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
21489,2,NaN,47,Среднее,1,женат / замужем,0,M,компаньон,0,NaN,сделка с автомобилем
21495,1,NaN,50,среднее,1,гражданский брак,1,F,сотрудник,0,NaN,свадьба
21497,0,NaN,48,ВЫСШЕЕ,0,женат / замужем,0,F,компаньон,0,NaN,строительство недвижимости
21502,1,NaN,42,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,строительство жилой недвижимости
21510,2,NaN,28,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,приобретение автомобиля


In [131]:
df[(df['days_employed'].isna()) ^ (df['total_income'].isna())].head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose


In [132]:
print('{:.2f}% строк имеют пропущенные значения '.format((sum(df['total_income'].isna() / df.shape[0])*100)))

10.10% строк имеют пропущенные значения 


##### Вывод: 
отсутствие значений в столбцах 'days_employed' и 'total_income' как-то зависят друг от друга, так как во всех строках в которых отсутствует одно из значений отсутствует и другое. Возможно, пропуски связанны с тем, что часть данных получены из другого источника где стаж и доход не заполняются.

Используем датафрейм `df` в котором заменим пропуски средним значением  для задач:
- поиска зависимости между наличием детей и возвратом кредита в срок
- поиска зависимости между семейным положением и возвратом кредита в срок

Используем датафрейм `df_cleared` в котором удалим строки с пропусками для задачи:
- поиска зависимости между уровнем дохода и возвратом кредита в срок

#### Заполним пропуски 'total_income' как среднее значение по группам 'income_type'.

In [133]:
mean_incomes = df[df['total_income'].notna()].groupby('income_type')['total_income'].mean()
for income_type, income  in zip(mean_incomes.index, mean_incomes):
    df.loc[(df['total_income'].isna()) & (df['income_type'] == income_type),'total_income'] = income

#### Создадим таблицу df_cleared в которой будут отсутствовать строки с пропущенными значениями 'total_income'

In [134]:
df_cleared = df.dropna()
print(df_cleared.shape)

(19351, 12)


#### Проверим есть ли отрицательные значения там где их быть не должно

In [135]:
clmns = ['children', 'days_employed', 'dob_years', 'total_income']

for c in clmns:
    count = df[df[c] < 0].shape[0]
    if count > 0:
        print('Количество строк с отрицательным значением', c, ':', count)
    else:
        print('Количество строк с отрицательным значением', c, ':', '-отсутствуют-')

Количество строк с отрицательным значением children : 47
Количество строк с отрицательным значением days_employed : 15906
Количество строк с отрицательным значением dob_years : -отсутствуют-
Количество строк с отрицательным значением total_income : -отсутствуют-


#### Узнаем распределение заёмщиков по количеству детей

In [136]:
df['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

'-1' ребёнок похоже не опечатку, '20' детей тоже скорее всего опечатка, так как нету заёмщиков с количеством детей больше 5

#### Исправим столбец со кол-вом детей. Заменим -1 на 1, а 20 на 2

In [137]:
df.loc[df['children'] == 20, 'children'] = 2
df.loc[df['children'] == -1, 'children'] = df.loc[df['children'] == -1, 'children'].abs()
print(df['children'].value_counts())

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64


#### Узнаем распределение заёмщиков по возрасту

In [138]:
df['dob_years'].value_counts().sort_index()

0     101
19     14
20     51
21    111
22    183
23    254
24    264
25    357
26    408
27    493
28    503
29    545
30    540
31    560
32    510
33    581
34    603
35    617
36    555
37    537
38    598
39    573
40    609
41    607
42    597
43    513
44    547
45    497
46    475
47    480
48    538
49    508
50    514
51    448
52    484
53    459
54    479
55    443
56    487
57    460
58    461
59    444
60    377
61    355
62    352
63    269
64    265
65    194
66    183
67    167
68     99
69     85
70     65
71     58
72     33
73      8
74      6
75      1
Name: dob_years, dtype: int64

Определим у каких типов заёмщиков возраст указан как '0'

In [139]:
df[df['dob_years'] == 0]['income_type'].value_counts()

сотрудник      55
пенсионер      20
компаньон      20
госслужащий     6
Name: income_type, dtype: int64

Заменять строки с пропущенным возрастом на среднее значение по всем строкам было бы неправильно, так как среди пропущенных значений, например, присутствуют пенсионеры, средний возраст которых определённо выше среднего возраста по всем группам. Поэтому заменим на средний возраст по типу занятости

In [140]:
df[df['dob_years'] != 0].groupby('income_type')['dob_years'].mean().astype('int')

income_type
безработный        38
в декрете          39
госслужащий        40
компаньон          39
пенсионер          59
предприниматель    42
сотрудник          40
студент            22
Name: dob_years, dtype: int64

In [141]:
mean_years = df[df['dob_years'] != 0].groupby('income_type')['dob_years'].mean().astype('int')
for income_type, dob_years  in zip(mean_years.index, mean_years):
    df.loc[(df['dob_years'] == 0) & (df['income_type'] == income_type),'dob_years'] = dob_years

In [142]:
print('Количество строк с нулевым стажем клиента:', df[df['dob_years'] == 0].shape[0])

Количество строк с нулевым стажем клиента: 0


#### Узнаем распределение заёмщиков по образованию

In [143]:
df['education'].value_counts().sort_index()

ВЫСШЕЕ                   274
Высшее                   268
НАЧАЛЬНОЕ                 17
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
Начальное                 15
Неоконченное высшее       47
СРЕДНЕЕ                  772
Среднее                  711
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
высшее                  4718
начальное                250
неоконченное высшее      668
среднее                13750
ученая степень             4
Name: education, dtype: int64

Приведём все значения к представлению строчными буквами

In [144]:
df['education'] = df['education'].str.lower()

In [145]:
df['education'].value_counts().sort_index()

высшее                  5260
начальное                282
неоконченное высшее      744
среднее                15233
ученая степень             6
Name: education, dtype: int64

####  Узнаем распределение заёмщиков по семейному положению

In [146]:
df['family_status'].value_counts()

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

В столбце 'family_status' пропусков и неправильных значений не замечено

#### Проверим значения в столбце 'days_employed' на наличие ошибок и аномалий

In [147]:
print('Количество строк с отрицательным значениям стажа:', df[df['days_employed'] < 0].shape[0])

Количество строк с отрицательным значениям стажа: 15906


In [148]:
print('Средний стаж в днях:', df[df['days_employed'] > 0]['days_employed'].mean())
print('Средний \"отрицательный\" стаж в днях:', df[df['days_employed'] < 0]['days_employed'].mean())
print('Количество строк с нулевым стажем:', df[df['days_employed'] == 0]['days_employed'].count())

Средний стаж в днях: 365004.30991626816
Средний "отрицательный" стаж в днях: -2353.015931998879
Количество строк с нулевым стажем: 0


15906 строк имеют отрицательный стаж, а для остальным строк средний стаж около 1000 лет. Нулевого стажа нет. Определить причину искажения данных не представляется возможным

In [149]:
print(f"Количество строк со стажем больше 10000 дней: {df[df['days_employed']>10000].shape[0]}")
print(f"Количество строк со стажем больше 300000 дней: {df[df['days_employed']>300000].shape[0]}")

Количество строк со стажем больше 10000 дней: 3445
Количество строк со стажем больше 300000 дней: 3445


In [150]:
df[df['days_employed']>300000]['income_type'].value_counts()

пенсионер      3443
безработный       2
Name: income_type, dtype: int64

Можно предположить, что модуль отрицательных значений является корректным стажем. А вот значения стажа более 300000 дней присущи пенсионерам, возможно стаж указан в часах. При рассмотрении такаго стажа как часы получается стаж около 40 лет, что похоже на правду для пенсионеров. Изменять значения не будем, так как не используем их задании.

In [151]:
hours_in_day = 24
df.loc[df['days_employed']>300000, 'days_employed'] = df.loc[df['days_employed']>300000, 'days_employed'].apply(lambda x: x/hours_in_day)
df.loc[df['days_employed']<0, 'days_employed']      = df.loc[df['days_employed']<0, 'days_employed'].apply(lambda x: x*(-1))

In [152]:
print('Новый средний стаж в днях:', df['days_employed'].mean())
print('Новый средний стаж в годах:', df['days_employed'].mean() / 365)

Новый средний стаж в днях: 4641.641176180646
Новый средний стаж в годах: 12.716825140220948


#### Заполним пропуски 'days_employed' как среднее значение по группам 'income_type'.

In [153]:
days = df[df['days_employed'].notna()].groupby('income_type')['days_employed'].mean()
for income_type, days_employed  in zip(days.index, days):
    df.loc[(df['days_employed'].isna()) & (df['income_type'] == income_type),'days_employed'] = days_employed

#### Проверим избавились ли мы от NaN и ошибочных значений

In [154]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [155]:
df[['children', 'days_employed', 'total_income']].mean()

children              0.479721
days_employed      4657.154094
total_income     167395.915741
dtype: float64

##### Вывод:
- избавились от пропущенных значений
- исправили ошибочные значения
- исправили нулевые значения

<a id="float_to_int"></a>
### 2.2 Замена вещественного типа данных на целочисленный
Для изменения float на int будем использовать метод astype()

In [79]:
try:
    df[['days_employed', 'total_income']] = df[['days_employed', 'total_income']].astype('int')
except:
    print('Что-то пошло не так: ', sys.exc_info()[0])
else:
    print("Значения в столбцах 'days_employed' и 'total_income' успешно заменены с вещественных на целочисленные")

Значения в столбцах 'days_employed' и 'total_income' успешно заменены с вещественных на целочисленные


<a id="duplicates"></a>
### 2.3 Обработка дубликатов

In [80]:
df[df.duplicated(keep=False)].sort_values('dob_years')

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
20297,1,2326,23,среднее,1,гражданский брак,1,F,сотрудник,0,161380,сыграть свадьбу
8853,1,2326,23,среднее,1,гражданский брак,1,F,сотрудник,0,161380,сыграть свадьбу
15892,0,2326,23,среднее,1,Не женат / не замужем,4,F,сотрудник,0,161380,сделка с подержанным автомобилем
19321,0,2326,23,среднее,1,Не женат / не замужем,4,F,сотрудник,0,161380,сделка с подержанным автомобилем
3452,0,2326,29,высшее,0,женат / замужем,0,M,сотрудник,0,161380,покупка жилой недвижимости
...,...,...,...,...,...,...,...,...,...,...,...,...
5865,0,15208,66,среднее,1,вдовец / вдова,2,F,пенсионер,0,137127,операции со своей недвижимостью
9528,0,15208,66,среднее,1,вдовец / вдова,2,F,пенсионер,0,137127,операции со своей недвижимостью
9604,0,15208,71,среднее,1,гражданский брак,1,F,пенсионер,0,137127,на проведение свадьбы
7938,0,15208,71,среднее,1,гражданский брак,1,F,пенсионер,0,137127,на проведение свадьбы


In [81]:
print('Количество дубликатов', df[df.duplicated()].shape[0])

Количество дубликатов 71


In [82]:
df.drop_duplicates(inplace=True)

In [83]:
if df[df.duplicated()].shape[0] == 0:
    print('Дубликаты успешно удалены')
else:
    print('Что-то пошло не так')

Дубликаты успешно удалены


В таблице нет уникальных айдишников, либо других идентификационных параметров. Такие дубликаты могут быть случаным совпадением.

##### Вывод
- Данные были предобработаны и оставшиеся полные дубликаты были удалены методом drop_duplicates(). Было бы целесообразно удалить дубликаты до расчёта средних значений. Не исключено появление дубликатов после обработки столбца 'purpose'.
- Появление дубликатов может быть связано с тем, что информация собиралась с разных источниках в которых какие-то данные совпадали

<a id="lemma"></a>
### 2.4 Выделение леммы в значениях столбца с целями получения кредита

In [84]:
df['purpose'].value_counts()

свадьба                                   791
на проведение свадьбы                     768
сыграть свадьбу                           765
операции с недвижимостью                  675
покупка коммерческой недвижимости         661
операции с жильем                         652
покупка жилья для сдачи                   651
операции с коммерческой недвижимостью     650
покупка жилья                             646
жилье                                     646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          624
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

In [85]:
m = Mystem()

purposes_all = set()
test = []
for purpose in df['purpose'].value_counts().index:
    for p in purpose.split():
        purposes_all.add(p)

In [86]:
lemmas = m.lemmatize(repr(purposes_all))
print(Counter(lemmas))

Counter({"', '": 47, 'жилье': 4, 'автомобиль': 4, 'образование': 3, 'свадьба': 3, 'свой': 3, 'недвижимость': 3, 'высокий': 3, 'подержанный': 2, 'дополнительный': 2, 'покупка': 2, "{'": 1, 'профильный': 1, 'с': 1, 'жилой': 1, 'проведение': 1, 'сделка': 1, 'семья': 1, 'сдача': 1, 'собственный': 1, 'для': 1, 'заниматься': 1, 'операция': 1, 'со': 1, 'строительство': 1, 'ремонт': 1, 'получение': 1, 'коммерческий': 1, 'сыграть': 1, 'приобретение': 1, 'на': 1, "'}\n": 1})


##### Вывод:
- Использовалась лемматизация целей кредита с помощью библиотеки pymystem3

<a id="categories"></a>
### 2.5 Категоризация данных

In [87]:
purpose_dict = df[['purpose']]
print(purpose_dict.head(5))

                      purpose
0               покупка жилья
1     приобретение автомобиля
2               покупка жилья
3  дополнительное образование
4             сыграть свадьбу


Оставим только уникальные цели кредита. Используем для этого метод drop_duplicates.

(можно было бы использовать pd.unique)

In [88]:
purpose_dict = purpose_dict.drop_duplicates().reset_index(drop=True)
print(purpose_dict)

                                   purpose
0                            покупка жилья
1                  приобретение автомобиля
2               дополнительное образование
3                          сыграть свадьбу
4                        операции с жильем
5                              образование
6                    на проведение свадьбы
7                  покупка жилья для семьи
8                     покупка недвижимости
9        покупка коммерческой недвижимости
10              покупка жилой недвижимости
11  строительство собственной недвижимости
12                            недвижимость
13              строительство недвижимости
14      на покупку подержанного автомобиля
15            на покупку своего автомобиля
16   операции с коммерческой недвижимостью
17        строительство жилой недвижимости
18                                   жилье
19         операции со своей недвижимостью
20                              автомобили
21                   заняться образованием
22        с

In [89]:
print('В оригинальном датасете', purpose_dict.shape[0], 'категорий целей кредита')

В оригинальном датасете 38 категорий целей кредита


#### Выделим категории целей кредита на основе результатов лемматизации

In [90]:
puproses_new = {1:'недвижимость', 2:'ремонт', 3:'автомобиль', 4:'образование', 5:'свадьба'}

Так как по результатам лемметизации видим, что одними из целей кредита являются строительство, жильё или недвижимость, то для определения цели кредита в категорию 'недвижимость' используем дополнительный словарь, чтобы объединить одинаковые по смыслу цели кредита

In [91]:
categories = {1:['недвижимость', 'строительство', 'жилье'], 2:['ремонт'], 3:['автомобиль'], 4:['образование'], 5:['свадьба']}

In [92]:
russian_stemmer = SnowballStemmer('russian') 
purpose_id = []
purpose_new = []

for i in purpose_dict['purpose']:
    for word in i.split():
        stemmed_word = russian_stemmer.stem(word)
        for i in categories:
            cat_id = 0
            if stemmed_word in russian_stemmer.stem(' '.join(categories[i])):
                cat_id = i
                break
        if cat_id != 0:
            purpose_id.append(cat_id)
            purpose_new.append(puproses_new[i])
            break

In [93]:
purpose_dict['id'] = purpose_id
purpose_dict['purpose_new'] = purpose_new
purpose_dict

,purpose,id,purpose_new
0,покупка жилья,1,недвижимость
1,приобретение автомобиля,3,автомобиль
2,дополнительное образование,4,образование
3,сыграть свадьбу,5,свадьба
4,операции с жильем,1,недвижимость
5,образование,4,образование
6,на проведение свадьбы,5,свадьба
7,покупка жилья для семьи,1,недвижимость
8,покупка недвижимости,1,недвижимость
9,покупка коммерческой недвижимости,1,недвижимость


#### Добавим категории целей кредита в исходный Датасет

In [94]:
def set_puprose_id(row):
    return purpose_dict.loc[purpose_dict['purpose'] == row['purpose'],'id'].iloc[0]

In [95]:
def set_puprose_new(row):
    return purpose_dict.loc[purpose_dict['purpose'] == row['purpose'],'purpose_new'].iloc[0]

In [96]:
df['purpose_id'] = df.apply(set_puprose_id, axis=1)
df['purpose_new'] = df.apply(set_puprose_new, axis=1)
df.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_id,purpose_new
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,1,недвижимость
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,3,автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,1,недвижимость
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,4,образование
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,5,свадьба


#### Категорируем месячный доход

Определим распределение доходов, чтобы понять как разбивать доходы по категориям

In [97]:
print('Минимальный доход:', df_cleared['total_income'].min())
print('Максимальный доход:', df_cleared['total_income'].max())
print('Средний доход:', df_cleared['total_income'].mean())
print('Медианный доход:', df_cleared['total_income'].median())

Минимальный доход: 20667.26379327158
Максимальный доход: 2265604.028722744
Средний доход: 167422.3022081719
Медианный доход: 145017.93753253992


Создадим функцию распределения по категориям в зависимости от доходов

In [98]:
total_income_categories = ['очень низкий доход', 'низкий доход', 'средний доход', 'высокий доход']
total_income = [0, 40000, 80000, 150000]

def get_income_category(income):
    if(len(total_income_categories) < len(total_income)):
        return 'ошибка'
    for i in reversed(range(len(total_income_categories))):
        if income > total_income[i]:
            return total_income_categories[i]
    return 'ошибка'

Создадим новый столбец 'total_income_category' с категориями дохода заёмщиков

In [158]:
df_cleared['total_income_category'] = df_cleared['total_income'].apply(get_income_category)

<ipython-input-158-67c0d8f34afd>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleared['total_income_category'] = df_cleared['total_income'].copy().apply(get_income_category)


Проверим распределение заёмщиков по категориям доходов

In [100]:
df_cleared['total_income_category'].value_counts()

высокий доход         9184
средний доход         7891
низкий доход          2151
очень низкий доход     125
Name: total_income_category, dtype: int64

##### Вывод:
- для целей кредита выделены новые категории: 'недвижимость', 'ремонт', 'автомобиль', 'образование', 'свадьба'
- для доходов выделены новые категории: 'очень низкий доход', 'низкий доход', 'средний доход', 'высокий доход'

<a id="answers"></a>
## 3. Ответы на вопросы исследования

Функция возвращающая отношение кол-ва заёмщиков с просроченной задолженностью к общему кол-ву заёмщиков, использую её для форматирования вывода

In [109]:
def get_percent_debt(a):
    return ('{:.2f}%'.format((sum(a) / len(a))*100))

<a id="answer_1"></a>
### 3.1 Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [110]:
data_pivot = df.pivot_table(index='children', values='debt', aggfunc=['count', 'sum', get_percent_debt])
data_pivot.columns = ['всего заёмщиков', 'кол-во должников', 'процент должников']
data_pivot.sort_values('процент должников')

,всего заёмщиков,кол-во должников,процент должников
children,,,
5,9,0,0.00%
0,14091,1063,7.54%
3,330,27,8.18%
1,4855,445,9.17%
2,2128,202,9.49%
4,41,4,9.76%


##### Вывод:
- лучше всего возвращают кредит заёмщики без детей (кол-во заёмщиков с 5 детьми всего 9, поэтому по ним вывод сделать нельзя)

<a id="answer_2"></a>
### 3.2 Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [103]:
data_pivot = df.pivot_table(index='family_status', values='debt', aggfunc=['count', 'sum', get_percent_debt])
data_pivot.columns = ['всего заёмщиков', 'число должников', 'процент должников']
data_pivot.sort_values('процент должников')

,всего заёмщиков,кол-во должников,процент должников
family_status,,,
вдовец / вдова,959,63,6.57%
в разводе,1195,85,7.11%
женат / замужем,12339,931,7.55%
гражданский брак,4151,388,9.35%
Не женат / не замужем,2810,274,9.75%


##### Вывод:
- лучше всего возвращают кредит вдовы
- хуже всего возвращают те кто не женат или в гражданском браке

<a id="answer_3"></a>
### 3.3 Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Для поиска зависимости между уровнем дохода и возвратом кредита в срок используем таблицу очищенную от пропущенных значений дохода

In [104]:
data_pivot = df_cleared.pivot_table(index='total_income_category', values='debt', aggfunc=['count', 'sum', get_percent_debt])
data_pivot.columns = ['всего заёмщиков', 'кол-во должников', 'процент должников']
data_pivot.sort_values('процент должников')

,всего заёмщиков,кол-во должников,процент должников
total_income_category,,,
низкий доход,2151,164,7.62%
высокий доход,9184,726,7.91%
очень низкий доход,125,10,8.00%
средний доход,7891,671,8.50%


##### Вывод:
- зависимости возврата кредита от уровня дохода нет

<a id="answer_4"></a>
### 3.4 Как разные цели кредита влияют на его возврат в срок?

In [105]:
data_pivot = df.pivot_table(index='purpose_new', values='debt', aggfunc=['count', 'sum', get_percent_debt])
data_pivot.columns = ['всего заёмщиков', 'кол-во должников', 'процент должников']
data_pivot.sort_values('процент должников')

,всего заёмщиков,кол-во должников,процент должников
purpose_new,,,
ремонт,607,35,5.77%
недвижимость,11145,848,7.61%
свадьба,2324,186,8.00%
автомобиль,3365,302,8.97%
образование,4013,370,9.22%


<a id="conclusion"></a>
## 4. Общий вывод
- хуже всего кредиты возвращают холостые или в гражданском браке
- чем больше детей в семье, тем больше вероятность перехода заёмщика в категорию должников
- для выявления влияния уровня дохода на возврат кредита нужно больше данных
- хуже всего вовращают кредиты на образование и автомобиль